PROC TABULATE
==================
F=: display with this format   
*: break down from parent  
var='newName': rename the var for display  
TABLES: if NO commas included in the TABLES statement. This indicates to SAS that this is to
be a one dimensional table.   

```r
PROC TABULATE DATA=report;
title bold "title"
VAR numericVar
CLASS categoryVar
TABLES (dimPage, dimRow, dimCol) or (dimRow, dimCol) or(dimCol)
RUN;
```

In [7]:

OPTIONS LS=64 PS=59 NOCENTER;

PROC FORMAT;  
   PICTURE PCT LOW-<0=' ' 0-HIGH='00000%';
RUN;

%LET NUMBER=5;
DATA SCORE;
   ARRAY ANS[&NUMBER] $ 1 ANS1-ANS&NUMBER; ***Student answers;
   ARRAY KEY[&NUMBER] $ 1 KEY1-KEY&NUMBER; ***Answer key;
   ARRAY S[&NUMBER] 3 S1-S&NUMBER; ***Score array 1=right,0=wrong;
   RETAIN KEY1-KEY&NUMBER; 

   ***Read the answer key;
   IF _N_ = 1 THEN INPUT (KEY1-KEY&NUMBER)($1.); 

   ***Read student responses;
   INPUT @1 ID 1-9  
         @11 (ANS1-ANS&NUMBER)($1.);

   ***Score the test;
   DO I = 1 TO &NUMBER;
      IF KEY[I] EQ ANS[I] THEN DO;
         S[I] = 1;
         ***Place an asterisk next to correct answer;
         SUBSTR(ANS[I],2,1) = '*'; 
      END;
      ELSE S[I] = 0;
   END;

   ***Compute Raw and Percentage scores;
   RAW = SUM (OF S1-S&NUMBER);  
   PERCENT = 100*RAW / &NUMBER;  

   KEEP ANS1-ANS&NUMBER S1-S&NUMBER KEY1-Key&NUMBER ID RAW PERCENT;

   LABEL ID = 'Social Security Number'
         RAW = 'Raw Score'
         PERCENT  = 'Percent Score';
DATALINES;  *** RUN if use INFILE 'C:\DATA';
ABCDE
123456789 ABCDE
035469871 BBBBB
111222333 ABCBE
212121212 CCCDE
867564733 ABCDA
876543211 DADDE
987876765 ABEEE
;
PROC SORT DATA=SCORE;
   BY ID;
RUN;


DATA TEMP;  
   SET SCORE;
   ARRAY ANS[*] $ 2 ANS1-ANS&NUMBER;
   DO QUESTION=1 TO &NUMBER;
      CHOICE=ANS[QUESTION];
      OUTPUT;
   END;
   KEEP QUESTION CHOICE PERCENT;
RUN;

PROC TABULATE DATA=TEMP;
   TITLE 'Item Analysis Using PROC TABULATE';
   CLASS QUESTION CHOICE;
   VAR PERCENT;
   TABLE QUESTION*CHOICE,
      PERCENT=' '*(PCTN<CHOICE>*F=PCT. MEAN*F=PCT.
      STD*F=10.2)   / RTS=20 MISSTEXT=' ';
   KEYLABEL ALL='Total' MEAN='Mean Score' PCTN='FREQ'
            STD='Standard Deviation';                   
RUN;


In [10]:
***Write correlation coefficients to a data set;
PROC CORR DATA=SCORE NOSIMPLE NOPRINT
          OUTP=CORROUT(WHERE = (_TYPE_='CORR'));
   VAR S1-S&NUMBER;
   WITH RAW;
RUN;

PROC PRINT DATA=CORROUT;
RUN;

Obs,_TYPE_,_NAME_,S1,S2,S3,S4,S5
1,CORR,RAW,0.79472,0.32646,0.79472,0.33113,0.32646


In [15]:


***Reshape the data set;
DATA CORR;
   SET CORROUT;
   ARRAY S[*] 3 S1-S&NUMBER;
   DO I=1 TO &NUMBER;
      CORR=S[I];
      OUTPUT;
   END;
   KEEP I CORR;
RUN;

***Compute quartiles;
PROC RANK DATA=SCORE GROUPS=4 OUT=QUART(DROP=PERCENT ID);
   RANKS QUARTILE;
   VAR RAW;
RUN;

***Create ITEM variable and reshape again;
DATA TAB;
   SET QUART;
   LENGTH ITEM $ 5 QUARTILE CORRECT I 3 CHOICE $ 1;
   ARRAY S[*] 3 S1-S&NUMBER;
   ARRAY ANS[*] $ 1 ANS1-ANS&NUMBER;
   ARRAY KEY[*] $ 1 KEY1-KEY&NUMBER;
   QUARTILE=QUARTILE+1;
   DO I = 1 TO &NUMBER;
      ITEM = RIGHT(PUT(I,3.)) || " " || KEY[I];
      CORRECT = S[I];
      CHOICE = ANS[I];
      OUTPUT;
   END;
   KEEP I ITEM QUARTILE CORRECT CHOICE;
RUN;

PROC SORT DATA=TAB;
   BY I;
RUN;

In [17]:

PROC SORT DATA=CORR;
   BY I;
RUN;

***Combine correlations and quartile information;
DATA BOTH;
   MERGE CORR TAB;
   BY I;
RUN;

***Print out a pretty table;
OPTIONS LS=72;
PROC TABULATE FORMAT=7.2 DATA=BOTH ORDER=INTERNAL NOSEPS;
   TITLE 'Item Statistics';
   LABEL QUARTILE = 'Quartile'
         CHOICE   = 'Choices';
   CLASS ITEM QUARTILE CHOICE;
   VAR CORRECT CORR;
   TABLE ITEM='#  Key'*F=6.,
   CHOICE*(PCTN<CHOICE>)*F=3. CORRECT=' '*MEAN='Diff.'*F=PERCENT5.2
      CORR=' '*MEAN='Corr.'*F=5.2
      CORRECT=' '*QUARTILE*MEAN='Prop. Correct'*F=PERCENT7.2 /
      RTS=8;
   KEYLABEL PCTN='%' ;
RUN;